In [7]:
# Import required libraries
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

def calculate_rsi(data, periods=14):
    """Calculate RSI for the given data"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_stochastic(data, k_period=14, d_period=3):
    """Calculate Stochastic Oscillator (K and D values)"""
    low_min = data['Low'].rolling(window=k_period).min()
    high_max = data['High'].rolling(window=k_period).max()
    
    k = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    d = k.rolling(window=d_period).mean()
    
    return k, d

def analyze_kd_signals(df):
    """Analyze K-D crossovers and next day price movements"""
    # Create signal columns
    df['K_Above_D'] = df['K'] > df['D']
    df['Next_Day_Open'] = df['Open'].shift(-1)
    df['Current_Close'] = df['Close']
    
    # Check if prediction was correct
    df['Signal_Correct'] = np.where(
        df['K_Above_D'],
        # When K > D, check if next day open > current close
        df['Next_Day_Open'] > df['Current_Close'],
        # When K < D, check if next day open < current close
        df['Next_Day_Open'] < df['Current_Close']
    )
    
    return df

# Set the stock symbol for JSW Energy
stock_symbol = 'JSWENERGY.NS'

# Calculate date range (2 years from today)
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)

# Create a Ticker object and download data
jsw = yf.Ticker(stock_symbol)
df = jsw.history(start=start_date, end=end_date)
df = df.reset_index()

# Calculate technical indicators
df['RSI'] = calculate_rsi(df['Close'])
df['K'], df['D'] = calculate_stochastic(df)

# Add other analysis columns
df['Daily_Return'] = df['Close'].pct_change() * 100
df['Moving_Avg_20'] = df['Close'].rolling(window=20).mean()
df['Moving_Avg_50'] = df['Close'].rolling(window=50).mean()

# Analyze K-D signals
df = analyze_kd_signals(df)

# Create interactive chart
fig = make_subplots(rows=5, cols=1, 
                    shared_xaxes=True,
                    vertical_spacing=0.05,
                    row_heights=[0.4, 0.15, 0.15, 0.15, 0.15])

# Add candlestick chart
fig.add_trace(go.Candlestick(x=df['Date'],
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name='OHLC'),
                            row=1, col=1)

# Add moving averages
fig.add_trace(go.Scatter(x=df['Date'], 
                        y=df['Moving_Avg_20'],
                        line=dict(color='orange', width=1),
                        name='20-day MA'),
                        row=1, col=1)

fig.add_trace(go.Scatter(x=df['Date'], 
                        y=df['Moving_Avg_50'],
                        line=dict(color='blue', width=1),
                        name='50-day MA'),
                        row=1, col=1)

# Add volume
fig.add_trace(go.Bar(x=df['Date'], 
                     y=df['Volume'],
                     name='Volume',
                     marker_color='rgba(128,128,128,0.5)'),
                     row=2, col=1)

# Add RSI
fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['RSI'],
                        line=dict(color='purple', width=1),
                        name='RSI'),
                        row=3, col=1)

# Add Stochastic Oscillator
fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['K'],
                        line=dict(color='blue', width=1),
                        name='%K'),
                        row=4, col=1)

fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['D'],
                        line=dict(color='red', width=1),
                        name='%D'),
                        row=4, col=1)

# Add Signal Accuracy
fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['Signal_Correct'].rolling(window=20).mean() * 100,
                        line=dict(color='green', width=1),
                        name='Signal Accuracy (20-day MA)'),
                        row=5, col=1)

# Update layout
fig.update_layout(
    title='JSW Energy Technical Analysis with K-D Signal Accuracy',
    yaxis_title='Stock Price (INR)',
    yaxis2_title='Volume',
    yaxis3_title='RSI',
    yaxis4_title='Stochastic',
    yaxis5_title='Signal Accuracy %',
    height=1500,
    showlegend=True,
    xaxis5_title='Date'
)

# Add reference lines
fig.add_hline(y=70, line_dash="dash", line_color="red", row=3)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=3)
fig.add_hline(y=80, line_dash="dash", line_color="red", row=4)
fig.add_hline(y=20, line_dash="dash", line_color="green", row=4)
fig.add_hline(y=50, line_dash="dash", line_color="gray", row=5)

# Show the interactive chart
fig.show()

# Calculate signal accuracy statistics
total_signals = df['Signal_Correct'].count()
correct_signals = df['Signal_Correct'].sum()
accuracy = (correct_signals / total_signals) * 100

# Display signal analysis
print("\nK-D Signal Analysis:")
print("="*100)
print(f"Total Trading Days: {total_signals}")
print(f"Correct Signals: {correct_signals}")
print(f"Overall Accuracy: {accuracy:.2f}%")

# Display recent signals
print("\nRecent K-D Signals and Results:")
print("="*100)
recent_signals = df[['Date', 'Close', 'Next_Day_Open', 'K', 'D', 'K_Above_D', 'Signal_Correct']].tail(10)
print(recent_signals)

# Save to CSV
df.to_csv('jsw_energy_kd_analysis.csv', index=False)
print("\nDetailed K-D analysis data has been saved to 'jsw_energy_kd_analysis.csv'")

# Calculate separate accuracies for K>D and K<D signals
k_above_d = df[df['K_Above_D']]
k_below_d = df[~df['K_Above_D']]

k_above_d_accuracy = (k_above_d['Signal_Correct'].sum() / len(k_above_d)) * 100
k_below_d_accuracy = (k_below_d['Signal_Correct'].sum() / len(k_below_d)) * 100

print("\nDetailed Signal Analysis:")
print("="*100)
print(f"K>D Signal Accuracy: {k_above_d_accuracy:.2f}%")
print(f"K<D Signal Accuracy: {k_below_d_accuracy:.2f}%")


K-D Signal Analysis:
Total Trading Days: 494
Correct Signals: 237
Overall Accuracy: 47.98%

Recent K-D Signals and Results:
                         Date  Close  Next_Day_Open     K     D  K_Above_D  \
484 2025-02-06 00:00:00+05:30 494.20         492.75 38.37 32.52       True   
485 2025-02-07 00:00:00+05:30 484.65         483.60 36.21 37.55      False   
486 2025-02-10 00:00:00+05:30 476.90         475.10 29.98 34.86      False   
487 2025-02-11 00:00:00+05:30 466.90         466.85 21.95 29.38      False   
488 2025-02-12 00:00:00+05:30 467.60         468.00 27.26 26.40       True   
489 2025-02-13 00:00:00+05:30 467.65         469.00 29.60 26.27       True   
490 2025-02-14 00:00:00+05:30 437.10         436.05  3.40 20.09      False   
491 2025-02-17 00:00:00+05:30 431.50         430.50 13.04 15.35      False   
492 2025-02-18 00:00:00+05:30 440.40         439.00 22.14 12.86       True   
493 2025-02-19 00:00:00+05:30 454.85            NaN 36.91 24.03       True   

     Signal_Corr